# Climate Change and Deaths by Natural Disasters

*Sebastian Fürndraht, Hannes Rokitte, Paul Schmitt, Lukas Wieser*

## Overview
- Introduction
    - Research Questions
    - Used Datasets
    - Requirements & Dependencies
    - Constants
    - Download Temperature Data
- Data Integration
    - ...
- Prepare Datasets
    - ...
- Data Exploration
    - ...
- Conclusion
    - ...

## 1. Introduction

### Research Question

### Used Datasets

### Requirements & Dependencies

This project was created using Python 3.9.
The exact versions of the dependencies can be installed with the following command.

In [1]:
#!pip install -r requirements.txt -q

In [2]:
import numpy as np
import pandas as pd
import urllib.parse
import requests
from pathlib import Path

### Constants

In [3]:
# Constants should be uppercase
# Path variables should end with FOLDER or FILE
RAW_TEMP_DATA_FOLDER = "data/raw/temperature/"
COUNTRIES_LIST_FILE = "temp-countries-list.csv"

DIS_RAW_FILE = Path('data/raw/disaster/emdat_public_2022_12_22_full.xlsx')
DIS_PROCESSED_ALL_FILE = Path("data/processed/disaster/disaster-all.csv")
DIS_PROCESSED_FOLDER = "data/processed/disaster"

TEMP_PROCESSED_FOLDER = "data/processed/temperature"
POP_PROCESSED_FOLDER = "data/processed/population"

### Create Directories

In [4]:
Path(DIS_PROCESSED_FOLDER).mkdir(parents=True, exist_ok=True)
Path(TEMP_PROCESSED_FOLDER).mkdir(parents=True, exist_ok=True)
Path(POP_PROCESSED_FOLDER).mkdir(parents=True, exist_ok=True)

### Download Temperature Data
Automatically download the regional and country temperature data, so we don't have to download each file by ourselves.

In [5]:
countries = pd.read_csv("data/raw/temperature/countries-list.csv", sep=";")

In [6]:
temp_regions = countries["Region"].dropna().unique().tolist()
temp_countries = countries["Country"].tolist()

In [7]:
def download_temperature_countries(country_names: list[str]):
    for country in country_names:
        print(f"downloading {country}")
        country_encoded = urllib.parse.quote(country.lower().replace(" ", "-"),encoding='cp1252')
        url = f"http://berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/{country_encoded}-TAVG-Trend.txt"
        response = requests.get(url)
        data = response.text
        with open(f'data/raw/temperature/countries-land/{country}.txt', 'w', encoding="utf-8") as file:
            file.write(data)

In [8]:
def download_temperature_regions(region_names: list[str]):
    for region in region_names:
        print(f"downloading {region}")
        region_encoded = urllib.parse.quote(region.lower().replace(" ", "-"),encoding='cp1252')
        url = f"http://berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/{region_encoded}-TAVG-Trend.txt"
        response = requests.get(url)
        data = response.text
        with open(f'data/raw/temperature/regions-land/{region}.txt', 'w', encoding="utf-8") as file:
            file.write(data)

Change the variable `should_download_temperature_data` to `True`, to download the temperature data of countries & regions. (This should not be necessary, since the data should already be downloaded)

In [9]:
should_download_temperature_data = False
if should_download_temperature_data:
    download_temperature_countries(temp_countries)
    download_temperature_regions(temp_regions)

## 2. Data Integration

### Load Datasets

In [10]:
disasters = pd.read_excel('data/raw/disaster/emdat_public_2022_12_22_full.xlsx', skiprows=6, sheet_name="emdat data")

temperature_countries = pd.read_csv("data/raw/temperature/countries-list.csv", sep=";")
population_by_country = pd.read_excel('data/raw/population/gapminder-population-v7.xlsx', sheet_name="data-for-countries-etc-by-year")
population_by_region = pd.read_excel('data/raw/population/gapminder-population-v7.xlsx', sheet_name="data-for-regions-by-year")

un_country_codes = pd.read_csv("data/raw/country-codes/un-country-codes.csv", sep=";")
cia_country_codes = pd.read_csv("data/raw/country-codes/cia-country-codes.csv", sep="\t")

C:\Users\lukas\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
dis = pd.read_excel(DIS_RAW_FILE, skiprows = 6)

Preprocess CIA dataset (since it is a bit messy)

In [12]:
cia_country_codes.head(3)

,Entity,GENC,ISO 3166,Stanag,Internet,Comment
0,Afghanistan,AFG,AF | AFG | 004,AFG,.af,-
1,Akrotiri,XQZ,- | - | -,-,-,-
2,Albania,ALB,AL | ALB | 008,ALB,.al,-


In [13]:
# split iso-codes in separate columns
cia_country_codes[["ISO-alpha2","ISO-alpha3","ISO-numeric"]] = cia_country_codes["ISO 3166"].str.split("|",2,expand=True)
cia_country_codes.drop(columns=["ISO 3166"], inplace=True)
# strip whitespaces from iso-codes
cia_country_codes[["ISO-alpha2","ISO-alpha3","ISO-numeric"]] = cia_country_codes[["ISO-alpha2","ISO-alpha3","ISO-numeric"]].apply(lambda x: x.str.strip())
# replace not existing iso-codes with NaN for more clarity
cia_country_codes["ISO-alpha2"].replace("-", np.nan, inplace=True)
cia_country_codes["ISO-alpha3"].replace("-", np.nan, inplace=True)
cia_country_codes["ISO-numeric"].replace("-", np.nan, inplace=True)
# show preprocessed cia data
cia_country_codes.head(3)

,Entity,GENC,Stanag,Internet,Comment,ISO-alpha2,ISO-alpha3,ISO-numeric
0,Afghanistan,AFG,AFG,.af,-,AF,AFG,004
1,Akrotiri,XQZ,-,-,-,NaN,NaN,NaN
2,Albania,ALB,ALB,.al,-,AL,ALB,008


### Determine ISO codes for temperature data

Remove aggregated countries.

In [14]:
temperature_countries.shape

(237, 2)

We can see, that e.g. Denmark appears twice. This issue happens multiple times, and is due to the reason that some countries are aggregates of other countries e.g. `Denmark` consists of `Denmark (Europe)` also known as `Denmark Mainland`, and `Greenland`. The bearkley earth website has a worldmap on which the country is highlighted, this helped us to better understand what each of the conflicting countries is.

In [15]:
temperature_countries.iloc[55: 55+5]

,Country,Region
55,Cyprus,Asia
56,Czech Republic,Europe
57,Denmark,North America
58,Denmark (Europe),Europe
59,Djibouti,Africa


We decided to remove the "aggregated" country. Here is a list of the aggregate countries we removed, their individual parts still exists in the dataset:
- Denmark (Denmark Mainland, Greenland)
- France (France Mainland, French Guiana, French Polynesia, French Southern and Antarctic Lands)
- Netherlands (Netherlands Mainland, Sint Maarten, Curaçao, Aruba)
- United Kingdom (United Kingdom + Oversea territories such as Montserrat, Bermuda)

In [16]:
temperature_countries_remove = pd.DataFrame({
    "Country": ["Denmark","France", "Netherlands", "United Kingdom"],
    "Region": ["North America", np.nan, "Europe", "Europe"]
})
temperature_countries_cleaned = pd.concat([temperature_countries, temperature_countries_remove]).drop_duplicates(keep=False)
temperature_countries_cleaned.shape

(233, 2)

Rename Countries & Match ISO Codes

In [17]:
# some countries need to be renamed so that we find the matching country-code later
new_country_names = {
    "Denmark (Europe)": "Denmark",
    "France (Europe)": "France",
    "Netherlands (Europe)": "Netherlands",
    "United Kingdom (Europe)": "United Kingdom",
    "Åland": "Åland Islands",
    "Czech Republic": "Czechia",
    "Turkey": "Türkiye",
    "Svalbard and Jan Mayen": "Svalbard and Jan Mayen Islands",
    "Cape Verde": "Cabo Verde",
    "Turks and Caicas Islands": "Turks and Caicos Islands",
    "Swaziland": "Eswatini",
    "Macedonia": "North Macedonia",
    "Côte d'Ivoire": "Côte d’Ivoire",
    "Federated States of Micronesia": "Micronesia (Federated States of)",
    "South Georgia and the South Sandwich Isla": "South Georgia and the South Sandwich Islands",
    "Bonaire, Saint Eustatius and Saba": "Bonaire, Sint Eustatius and Saba",
    "Congo (Democratic Republic of the)": "Democratic Republic of the Congo",
    "South Korea": "Korea, South",
    "North Korea": "Korea, North",
    "Palestina": "State of Palestine"
}

temperature_countries_cleaned = temperature_countries_cleaned.replace({"Country": new_country_names}, inplace=False)

# left-join cia-country-codes and un-country-codes
temperature_countries_with_iso = temperature_countries_cleaned.merge(cia_country_codes,how="left",left_on='Country', right_on='Entity')[["Country","ISO-alpha3"]]
temperature_countries_with_iso = temperature_countries_with_iso.merge(un_country_codes,how="left",left_on='Country', right_on='Country or Area')[["Country","ISO-alpha3", "ISO-alpha3 Code"]]

# fill missing cia-country codes with un-country-codes
temperature_countries_with_iso["ISO-alpha3"].fillna(temperature_countries_with_iso["ISO-alpha3 Code"], inplace=True)
temperature_countries_with_iso.drop(columns=["ISO-alpha3 Code"], inplace=True)

# show countries for which we could not find an ISO code
temperature_countries_with_iso[temperature_countries_with_iso["ISO-alpha3"].isna()]

,Country,ISO-alpha3
18,Baker Island,NaN
113,Kingman Reef,NaN
161,Palmyra Atoll,NaN


These 3 countries/areas do not have any country codes in general, and are quite small, so we just ignore them later on.

In [18]:
# Todo: When combining adjust to use variable without saving temperature_countries_with_iso as csv file
processed_countries_list_path = 'data/processed/temperature/temp-countries-list.csv'
temperature_countries_with_iso.to_csv(processed_countries_list_path, index=False)

### Which countries are in which datasets?

Disaster vs. Temperature Dataset

In [19]:
berkely_iso_codes = set(temperature_countries_with_iso["ISO-alpha3"].dropna().tolist())
emdat_iso_codes = set(disasters["ISO"].unique().tolist())

emdat_and_bekely = emdat_iso_codes.intersection(berkely_iso_codes)
emdat_without_berkely = emdat_iso_codes-emdat_and_bekely
berkely_without_emdat = berkely_iso_codes-emdat_and_bekely

print(f"countries in emdat & berkely: {len(emdat_and_bekely)}")
print(f"countries in emdat but not berkely ({len(emdat_without_berkely)}):")
print(sorted(emdat_without_berkely))
print(f"countries in berkely but not emdat ({len(berkely_without_emdat)}):")
print(sorted(berkely_without_emdat))

countries in emdat & berkely: 209
countries in emdat but not berkely (22):
['ANT', 'AZO', 'BMU', 'BRN', 'COK', 'CSK', 'DDR', 'DFR', 'MDV', 'MHL', 'SCG', 'SHN', 'SPI', 'SSD', 'SUN', 'TKL', 'TUV', 'VUT', 'WLF', 'YMD', 'YMN', 'YUG']
countries in berkely but not emdat (20):
['ABW', 'ALA', 'AND', 'ATA', 'ATF', 'BES', 'CXR', 'ESH', 'FLK', 'FRO', 'GGY', 'GRL', 'HMD', 'JEY', 'LIE', 'MCO', 'SGS', 'SJM', 'SMR', 'SPM']


The Countries for which we have disaster data, but no temperature data are as follows:
- Existing Countries (usually very small countries/islands):
    - AZO Azores Islands
    - BMU Bermuda
    - BRN Brunei Darussalam
    - COK Cook Islands (the)
    - MDV Maldives
    - MHL Marshall Islands (the)
    - SHN Saint Helena, Ascension and Tristan da Cunha
    - SSD South Sudan
    - TKL Tokelau
    - TUV Tuvalu
    - VUT Vanuatu
    - WLF Wallis and Futuna
- Existing Countries (but invalid country code):
    - SPI Canary Islands
- Former Countries:
    - ANT Netherlands Antilles
    - CSK Czechoslovakia
    - DDR Germany Dem Rep
    - DFR Germany Fed Rep
    - SCG Serbia Montenegro
    - SUN Soviet Union
    - YMD Yemen P Dem Rep
    - YMN Yemen Arab Rep
    - YUG Yugoslavia

Disaster vs Population Dataset

In [20]:
gapminder_iso_codes = set(population_by_country["geo"].str.upper().unique())

In [21]:
emdat_and_gapminder = emdat_iso_codes.intersection(gapminder_iso_codes)
emdat_without_gapminder = emdat_iso_codes-emdat_and_gapminder
gapminder_without_emdat = gapminder_iso_codes-emdat_and_gapminder

print(f"countries in emdat & gapminder: {len(emdat_and_gapminder)}")
print(f"countries in emdat but not gapminder ({len(emdat_without_gapminder)}):")
print(sorted(emdat_without_gapminder))
print(f"countries in gapminder but not emdat ({len(gapminder_without_emdat)}):")
print(sorted(gapminder_without_emdat))

countries in emdat & gapminder: 191
countries in emdat but not gapminder (40):
['AIA', 'ANT', 'ASM', 'AZO', 'BLM', 'BMU', 'COK', 'CSK', 'CUW', 'CYM', 'DDR', 'DFR', 'GLP', 'GUF', 'GUM', 'IMN', 'MAC', 'MAF', 'MNP', 'MSR', 'MTQ', 'MYT', 'NCL', 'NIU', 'PRI', 'PYF', 'REU', 'SCG', 'SHN', 'SPI', 'SUN', 'SXM', 'TCA', 'TKL', 'VGB', 'VIR', 'WLF', 'YMD', 'YMN', 'YUG']
countries in gapminder but not emdat (6):
['AND', 'HOS', 'LIE', 'MCO', 'NRU', 'SMR']


The Countries for which we have disaster data, but no population data are as follows:
- Existing Countries (independent)
    - COK Cook Islands (the)
    - NIU Niue
- Existing Countries (dependent e.g .oversea territories)
    - AIA Anguilla
    - ASM American Samoa
    - AZO Azores Islands
    - BLM Saint Barthélemy
    - BMU Bermuda
    - CUW Curaçao
    - CYM Cayman Islands (the)
    - GLP Guadeloupe
    - GUF French Guiana
    - GUM Guam
    - IMN Isle of Man
    - MAC Macao
    - MAF Saint Martin (French Part)
    - MNP Northern Mariana Islands (the)
    - MSR Montserrat
    - MTQ Martinique
    - MYT Mayotte
    - NCL New Caledonia
    - PRI Puerto Rico
    - PYF French Polynesia
    - REU Réunion
    - SHN Saint Helena, Ascension and Tristan da Cunha
    - SPI Canary Islands
    - SXM Sint Maarten (Dutch part)
    - TCA Turks and Caicos Islands (the)
    - TKL Tokelau
    - VGB Virgin Island (British)
    - VIR Virgin Island (U.S.)
    - WLF Wallis and Futuna
- Former Countries
    - ANT Netherlands Antilles
    - CSK Czechoslovakia
    - DDR Germany Dem Rep
    - DFR Germany Fed Rep
    - SCG Serbia Montenegro
    - SUN Soviet Union
    - YMD Yemen P Dem Rep
    - YMN Yemen Arab Rep
    - YUG Yugoslavia

### Which regions are in which datasets?

In [22]:
disasters["Continent"].unique().tolist()

['Africa', 'Asia', 'Europe', 'Americas', 'Oceania']

In [23]:
population_by_region["geo"].unique().tolist()

['africa', 'asia', 'europe', 'americas']

In [24]:
temperature_countries["Region"].dropna().unique().tolist()

['Asia', 'Europe', 'Africa', 'South America', 'Oceania', 'North America']

As we can see, each dataset has a different number of regions. Additionally, we don't know which countries belong to each region. For example the region Europe could consist of different countries the disaster dataset than in the temperature dataset. That's why we decided to compute the regional data ourselves, by aggregating the countries according to UN Regions.

### Which countries are need to be manually assigned a region

Check which countries are not in the UN Dataset, and thus need to be manually assigned a region:

In [25]:
un_iso_codes = set(un_country_codes["ISO-alpha3 Code"].tolist())

emdat_without_un = emdat_iso_codes-un_iso_codes
gapminder_without_un = gapminder_iso_codes-un_iso_codes
berkely_without_un = berkely_iso_codes-un_iso_codes

print(f"countries in emdat, but not un ({len(emdat_without_un)}): \n{emdat_without_un}")
print(f"countries in gapminder, but not un ({len(gapminder_without_un)}): \n{gapminder_without_un}")
print(f"countries in berkely, but not un ({len(berkely_without_un)}): \n{berkely_without_un}")

countries in emdat, but not un (12): 
{'DDR', 'TWN', 'DFR', 'CSK', 'ANT', 'SCG', 'YMN', 'YMD', 'SPI', 'SUN', 'AZO', 'YUG'}
countries in gapminder, but not un (2): 
{'HOS', 'TWN'}
countries in berkely, but not un (1): 
{'TWN'}


## 3. Prepare Datasets

### 3.1 Disaster Data

Task:

The goal is to convert the data into the following formats for later use.
Along the way, this notebook does some data-preparation


Disaster-All
disaster/disaster-all:
Columns: disaster_no, year, subgroup, type, total_deaths, dis_mag_value, dis_mag_scale, start_year, end_year
Other interesting columns?

Prefix: dis


Publisher: Centre for Research on the Epidemiology of Disasters (CRED)

CRED defines a disaster as “a situation or event that overwhelms local capacity, necessitating a
request at the national or international level for external assistance; an unforeseen and often sudden
event that causes great damage, destruction and human suffering”

For a disaster to be entered into the database at least one of the following criteria must be fulfilled:

- 10 or more people reported killed
- 100 or more people reported affected
- Declaration of a state of emergency
- Call for international assistance

First look at the disaster data

In [27]:
dis.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
2,1901-0003-BEL,1901,3,NaN,Technological,Technological,Industrial accident,Explosion,NaN,Coal mine,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
3,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN
4,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN


#### Select & Rename Attributes

1. Replace whitespaces with underscores
2. Convert every character to lowercase
3. Rename specific columns to ensure uniformity

In [32]:
# Remove whitespaces from all col-names and convert them to lower-case
dis.columns = [c.replace(' ', '_').lower() for c in dis.columns]
dis.rename(columns={'country':'country_name', 'iso':'country_code', 'disaster_group':'group','disaster_subgroup':'subgroup', 'disaster_subtype':'subtype', 'disaster_type':'type', 'total_deaths':'deaths'}, inplace=True)

In [50]:
# Select the most interesting columns
dis_all_col_names = ["year", "dis_no", "country_name", "country_code", "location","group","subgroup", "type", "subtype", "deaths", "dis_mag_value", "dis_mag_scale", "start_year", "end_year"]
dis_all = dis.filter(items=dis_all_col_names)

#### Which disaster groups are present in the dataset ?

There are Natural disasters, technological disasters as well as complex disasters that represent specific events (e.g. famine) which are not directly linked to a natural hazard.

In [51]:
dis_all["group"].unique()

array(['Natural', 'Technological', 'Complex Disasters'], dtype=object)

We only focus on disasters which have a natural causation

In [52]:
dis_all = dis[dis_all["group"] == "Natural"].copy()
dis_all.drop(columns="group", inplace=True)

Which types of natural disasters are there ?

In [54]:
dis_all.groupby(["subgroup","type"]).agg({"deaths":"sum"}).reset_index()

,subgroup,type,deaths
0,Biological,Animal accident,12.0
1,Biological,Epidemic,9618804.0
2,Biological,Insect infestation,0.0
3,Climatological,Drought,11733889.0
4,Climatological,Glacial lake outburst,262.0
5,Climatological,Wildfire,4653.0
6,Extra-terrestrial,Impact,0.0
7,Geophysical,Earthquake,2343912.0
8,Geophysical,Mass movement (dry),4644.0
9,Geophysical,Volcanic activity,86893.0


The types of disasters are mostly the ones a normal person would expect when thinking about natural disasters. But there are some strange types like insect-infestations or animal-accident which are not that obvious to understand, they also have basically no deaths. Also for our research we want to exclude Epidemics since it would go beyond the scope of this task.

Therefore also decided to omit disasters of the subgroups `Biological` and `Extra-terrestrial`.

In [55]:
dis_all = dis_all[(dis_all["subgroup"] != "Biological") & (dis_all["subgroup"] != "Extra-terrestrial")]

We only consider the following types of disasters

In [56]:
dis_all["type"].unique()

array(['Drought', 'Earthquake', 'Volcanic activity',
       'Mass movement (dry)', 'Storm', 'Flood', 'Landslide', 'Wildfire',
       'Extreme temperature ', 'Fog', 'Glacial lake outburst'],
      dtype=object)

#### Handle Missing Values

Fill missing Values for the number of deaths

We can assume that missing values for the number of deaths of a particular disaster means that the deathtoll was 0.

For the subtype we take a look for which type of natural disasters a subtype is not provided.

In [57]:
dis_all.isna().sum()

dis_no                                           0
year                                             0
seq                                              0
glide                                        13353
subgroup                                         0
type                                             0
subtype                                       3116
disaster_subsubtype                          13768
event_name                                   12191
country_name                                     0
country_code                                     0
region                                           0
continent                                        0
location                                      1449
origin                                       10901
associated_dis                               11341
associated_dis2                              14123
ofda_response                                13343
appeal                                       13025
declaration                    

In [58]:
dis_all[dis_all["subtype"].isna()]["type"].unique()

array(['Flood', 'Storm', 'Landslide', 'Wildfire', 'Fog',
       'Mass movement (dry)', 'Volcanic activity', 'Drought',
       'Earthquake', 'Glacial lake outburst'], dtype=object)

Unfortunately the missing values in the subtype column do not correspond to specific types of disasters.
We can not conclude that easily what caused the values to be missing.

In [59]:
dis_all[dis_all["type"] == "Drought"].subtype.unique()

array(['Drought', nan], dtype=object)

Disasters of type drought have no further more specific subtype.
Consequently, we assign all droughts with missing values for subtype, the general type

In [60]:
dis_all.loc[dis_all["type"] == "Drought", "subtype"] = dis_all.loc[dis_all["type"] == "Drought", "subtype"].fillna("Drought")

For all other types, we unfortunately can not guess the subtype.
Therefore, we assign a custom label "Uncategorized" + the type of the disaster for each missing subtype
We do not need to do this for disasters of type extreme temperature, since subtypes for all observations are specified

In [61]:
dis_all.loc[dis_all["type"] == "Earthquake", "subtype"] = dis_all.loc[dis_all["type"] == "Earthquake", "subtype"].fillna("Uncategorized_Earthquake")
dis_all.loc[dis_all["type"] == "Storm", "subtype"] = dis_all.loc[dis_all["type"] == "Storm", "subtype"].fillna("Uncategorized_Storm")
dis_all.loc[dis_all["type"] == "Flood", "subtype"] = dis_all.loc[dis_all["type"] == "Flood", "subtype"].fillna("Uncategorized_Flood")

We assume that disasters with no death toll reported have a death toll of 0.
This also aligns with the information we get from emdat (deaths < 10 are missing)

In [62]:
dis_all[['deaths']] = dis_all[['deaths']].fillna(value=0)

#### Determine Regions

As mentioned in `data integration` we want to compute the region of each disaster, by taking the UN Region that is assigned to each Country, in which the disaster occurred.
Some country codes are not in the list of UN countries, thus we handle them specifically.

In [63]:
dis_iso_codes = set(dis_all["country_code"].unique())
un_iso_codes = set(un_country_codes["ISO-alpha3 Code"].tolist())
emdat_without_un = dis_iso_codes-un_iso_codes
print(f"countries in emdat, but not un ({len(emdat_without_un)}): \n{emdat_without_un}")

countries in emdat, but not un (12): 
{'DDR', 'TWN', 'DFR', 'CSK', 'ANT', 'SCG', 'YMN', 'YMD', 'SPI', 'SUN', 'AZO', 'YUG'}


##### Automatically assign regions with UN Dataset

For countries that were split in the past, but are now unified, we can just assign the unified country.

In [64]:
# Germany
dis_all.loc[dis_all['country_code'] == "DFR",'country_code'] = "DEU"
dis_all.loc[dis_all['country_code'] == "DDR",'country_code'] = "DEU"
# Yemen
dis_all.loc[dis_all['country_code'] == "YMD",'country_code'] = "YEM"
dis_all.loc[dis_all['country_code'] == "YMN",'country_code'] = "YEM"

Next we determine the region by using the un dataset:

In [65]:
dis_all = pd.merge(dis_all, un_country_codes[["ISO-alpha3 Code","Region Name", "Region Code"]], left_on='country_code', right_on='ISO-alpha3 Code', how='left')
dis_all.rename(columns={"Region Code": "region_code", "Region Name": "region_name"}, inplace=True)
dis_all.drop(columns=["ISO-alpha3 Code"],inplace=True)

##### Manually Assign Regions

Some countries clearly belong to one region, so we can assign the disasters manually

In [66]:
# Taiwan
dis_all.loc[dis_all['country_code']=='TWN','region_name'] = 'Asia'
dis_all.loc[dis_all['country_code']=='TWN','region_code'] = 142
# Czechoslovakia
dis_all.loc[dis_all['country_code']=='CSK','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='CSK','region_code'] = 150
# Yugoslavia
dis_all.loc[dis_all['country_code']=='YUG','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='YUG','region_code'] = 150
# Serbia Montenegro
dis_all.loc[dis_all['country_code']=='SCG','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='SCG','region_code'] = 150
# Netherlands Antilles
dis_all.loc[dis_all['country_code']=='ANT','region_name'] = 'Americas'
dis_all.loc[dis_all['country_code']=='ANT','region_code'] = 19
# Azores Islands
dis_all.loc[dis_all['country_code']=='AZO','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='AZO','region_code'] = 150
# Canary Islands
dis_all.loc[dis_all['country_code']=='SPI','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='SPI','region_code'] = 150

##### Manually Assign Regions of Disasters Soviet Union

For the Soviet Union disasters can occur in the european and/or asian parts.
Thus we also take the "location" attribute into account and try to derive if the disaster was in europe or asia.

In [67]:
mask_europe = dis_all.loc[dis_all['country_code']=='SUN']["location"].str.contains("Russian Federation|Ukraine|Moldavia|Siberia").fillna(False)
mask_asia = dis_all.loc[dis_all['country_code']=='SUN']["location"].str.contains("Kazakhstan|Azerbaijan|Uzbekistan|Turkmenistan|Georgia|Armenia|Kyrgystan|Tajikistan|Tajiskistan|Tadzhikistan|Tadjikistan|Caucasus region|Dushanbe", case=False).fillna(False)

In [68]:
dis_all[dis_all['country_code']=='SUN'][mask_europe & mask_asia]

,dis_no,year,seq,glide,subgroup,type,subtype,disaster_subsubtype,event_name,country_name,...,"insured_damages,_adjusted_('000_us$)",total_damages_('000_us$),"total_damages,_adjusted_('000_us$)",cpi,adm_level,admin1_code,admin2_code,geo_locations,region_name,region_code
849,1921-9001-SUN,1921,9001,NaN,Climatological,Drought,Drought,NaN,NaN,Soviet Union,...,NaN,NaN,NaN,6.597283,NaN,NaN,NaN,NaN,NaN,NaN


Only one Event happened in both the asian as well as the european part of the soviet union.
It is also a major event since it is a drought which caused the death of 1.2 million people.

Researching the details of this event one can conclude that this observation can only be the Russian famine of 1921–1922.
It mostly affected people living in europe, hence we assign this single observation the region europe.
(https://en.wikipedia.org/wiki/Russian_famine_of_1921%E2%80%931922)

For all other observations, the region should be unambiguous.

In [69]:
dis_all.loc[(dis_all['country_code']=='SUN') & mask_europe, "region_name"] = "Europe"
dis_all.loc[(dis_all['country_code']=='SUN') & mask_europe, "region_code"] = 150

dis_all.loc[(dis_all['country_code']=='SUN') & mask_asia, "region_name"] = "Asia"
dis_all.loc[(dis_all['country_code']=='SUN') & mask_asia, "region_code"] = 142

dis_all.loc[dis_all['dis_no']=='1921-9001-SUN', "region_name"] = "Europe"
dis_all.loc[dis_all['dis_no']=='1921-9001-SUN', "region_code"] = 150

The only disasters without a region are now these 3 in the soviet Union. However, since they have no death count we can safely ignore them.

In [70]:
dis_all

,dis_no,year,seq,glide,subgroup,type,subtype,disaster_subsubtype,event_name,country_name,...,"insured_damages,_adjusted_('000_us$)",total_damages_('000_us$),"total_damages,_adjusted_('000_us$)",cpi,adm_level,admin1_code,admin2_code,geo_locations,region_name,region_code
0,1900-9002-CPV,1900,9002,NaN,Climatological,Drought,Drought,NaN,NaN,Cabo Verde,...,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN,Africa,2.0
1,1900-9001-IND,1900,9001,NaN,Climatological,Drought,Drought,NaN,NaN,India,...,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN,Asia,142.0
2,1902-0012-GTM,1902,12,NaN,Geophysical,Earthquake,Ground movement,NaN,NaN,Guatemala,...,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN,Americas,19.0
3,1902-0003-GTM,1902,3,NaN,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN,Americas,19.0
4,1902-0010-GTM,1902,10,NaN,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Guatemala,...,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN,Americas,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14866,2022-0788-ZAF,2022,788,NaN,Hydrological,Flood,Flash flood,NaN,NaN,South Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Africa,2.0
14867,2022-0313-ZAF,2022,313,NaN,Hydrological,Flood,Uncategorized_Flood,NaN,NaN,South Africa,...,NaN,NaN,NaN,NaN,1,77311,NaN,KwaZulu-Natal (Adm1).,Africa,2.0
14868,2022-0356-ZAF,2022,356,NaN,Hydrological,Flood,Uncategorized_Flood,NaN,NaN,South Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Africa,2.0
14869,2022-0209-COD,2022,209,NaN,Meteorological,Storm,Convective storm,NaN,NaN,Congo (the Democratic Republic of the),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Africa,2.0


#### Save File

In [71]:
dis_all.to_csv(DIS_PROCESSED_ALL_FILE, index=False)

### 3.2 Population Data

In [72]:
'''
Population data

The population dataset by gapminder is a dataset containing information about the worlds population from 1800 until 2100.
The data is a composition from different sources. There a two main sources - a dataset by Angus Maddison and the CLIO Infra Project -  and the World Population Prospects (WPP) provided by the UN.
The dataset by Angus Maddison provides the data for the years 1800 - 1950. Population data after 1950 was taken from the WPP dataset.
Additional sources were used, to fill missing values for years or regions. A details list of sources can be found in the documentation. (https://www.gapminder.org/data/documentation/gd003/)
The primary data from the sources originates from census, informal census, indirect estimate and arbitrary guesses.

Modifications and Estimations:
- Summations of parts
- Larger area minus non-included parts
- Geographical interpolation
- Geographical extrapolation
- Temporal interpolation
- Temporal extrapolation
- Adjustments for under-enumeration
- Recalculated to fit present borders

Since the data for the year 1950 of the two main datasets did not match for every country, small adjustments and smoothing were applied.

Preprocessing:
For our purposes, the subdatasets "data-for-world-by-year", "data-for-regions-by-year" and "data-for-countries-by-year" are relevant.
We consider the period from 1900 until 2021 and remove redundant rows. We do this because the data after 2021 are based on estimated values.

There are no missing values.
Since the regions of the dataset did not match with the regions of our other datasets, we chose to compute the regions-population according to the regions from the un-country-codes.csv file.

Subdataset:
    population-global.csv
        Columns: year, population

    population-region.csv
        Columns: region_code, region_name, year, population

    population-country.csv
        Columns: country_code, country_name, year, population
'''

'\nPopulation data\n\nThe population dataset by gapminder is a dataset containing information about the worlds population from 1800 until 2100.\nThe data is a composition from different sources. There a two main sources - a dataset by Angus Maddison and the CLIO Infra Project -  and the World Population Prospects (WPP) provided by the UN.\nThe dataset by Angus Maddison provides the data for the years 1800 - 1950. Population data after 1950 was taken from the WPP dataset.\nAdditional sources were used, to fill missing values for years or regions. A details list of sources can be found in the documentation. (https://www.gapminder.org/data/documentation/gd003/)\nThe primary data from the sources originates from census, informal census, indirect estimate and arbitrary guesses.\n\nModifications and Estimations:\n- Summations of parts\n- Larger area minus non-included parts\n- Geographical interpolation\n- Geographical extrapolation\n- Temporal interpolation\n- Temporal extrapolation\n- Adju

In [73]:
pop_dict = pd.read_excel('data/raw/population/gapminder-population-v7.xlsx', sheet_name=['data-for-world-by-year', 'data-for-regions-by-year', 'data-for-countries-etc-by-year'])

#### Preprocess global data

In [74]:
pop_global_df = pop_dict.get('data-for-world-by-year')

print('raw data for world population')
print(pop_global_df.head(3))

# remove unnecessary columns
pop_global_df = pop_global_df[['time', 'Population']]
pop_global_df.rename(columns={'Population': 'population', 'time': 'year'}, inplace=True)
pop_global_df.set_index('year')

print('extract population data from 1900 until now')
pop_global_df = pop_global_df[pop_global_df['year'] >= 1900]
pop_global_df = pop_global_df[pop_global_df['year'] <= 2021]
pop_global_df['population'] = pop_global_df['population'].astype(int)
print(pop_global_df)

pop_global_df.to_csv('data/processed/population/population-global.csv', sep=';', index=False, header=True)

raw data for world population
     geo   name    time   Population
0  world  world  1800.0  985083734.9
1  world  world  1801.0  988518009.0
2  world  world  1802.0  991993182.0
extract population data from 1900 until now
       year  population
100  1900.0  1627123965
101  1901.0  1639684222
102  1902.0  1652898195
103  1903.0  1666760587
104  1904.0  1680799825
..      ...         ...
217  2017.0 -2147483648
218  2018.0 -2147483648
219  2019.0 -2147483648
220  2020.0 -2147483648
221  2021.0 -2147483648

[122 rows x 2 columns]


C:\Users\lukas\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


#### Prepare dataset for countries and regions

In [75]:
pop_country_df = pop_dict.get('data-for-countries-etc-by-year')
un_country_codes = pd.read_csv("data/raw/country-codes/un-country-codes.csv", sep=";")
un_country_codes = un_country_codes[['Region Code', 'Region Name', 'ISO-alpha3 Code']]

# extract population data from 1900 until now
pop_country_df = pop_country_df[pop_country_df['time'] >= 1900]
pop_country_df = pop_country_df[pop_country_df['time'] <= 2021]

# format column that will be joined
pop_country_df['geo'] = pop_country_df['geo'].str.upper()

# merge country-population and un-country-codes
pop_country_region_df = pd.merge(pop_country_df, un_country_codes, how='left', left_on='geo', right_on='ISO-alpha3 Code')

pop_country_region_df.rename(columns={'Region Code': 'region_code', 'Region Name': 'region_name', 'time': 'year', 'Population': 'population', 'geo': 'country_code', 'name': 'country_name'}, inplace=True)

# add missing region code to holy see and taiwan
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'HOS', 'region_code'] = 150
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'HOS', 'region_name'] = 'Europe'

pop_country_region_df.loc[pop_country_region_df['country_code'] == 'TWN', 'region_code'] = 142
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'TWN', 'region_name'] = 'Asia'

# remove unused column 'ISO-alpha3 Code'
pop_country_region_df.drop('ISO-alpha3 Code', axis=1, inplace=True)

# check if we worked correctly
print(pop_country_region_df[pop_country_region_df.isnull().any(axis=1)])

# the remaining missing values are population data for the country 'holy see' since the missing population is in the range < 1000 we, accept the missing values and calculate with the latest documented value.

     country_code country_name    year  population  region_code region_name
8885          HOS     Holy See  2001.0         NaN        150.0      Europe
8886          HOS     Holy See  2002.0         NaN        150.0      Europe
8887          HOS     Holy See  2003.0         NaN        150.0      Europe
8888          HOS     Holy See  2004.0         NaN        150.0      Europe
8889          HOS     Holy See  2005.0         NaN        150.0      Europe
8890          HOS     Holy See  2006.0         NaN        150.0      Europe
8891          HOS     Holy See  2007.0         NaN        150.0      Europe
8892          HOS     Holy See  2008.0         NaN        150.0      Europe
8893          HOS     Holy See  2009.0         NaN        150.0      Europe
8894          HOS     Holy See  2010.0         NaN        150.0      Europe
8895          HOS     Holy See  2011.0         NaN        150.0      Europe
8896          HOS     Holy See  2012.0         NaN        150.0      Europe
8897        

#### Save population regions dataset

In [76]:
columns_regions = ['region_code', 'region_name', 'country_code', 'country_name', 'year', 'population']

# select columns
pop_regions_df = pop_country_region_df[columns_regions]
pop_regions_df = pop_regions_df.groupby(['region_code', 'region_name', 'year'], as_index=False)['population'].sum()
pop_regions_df['population'] = pop_regions_df['population'].astype(int)

#store in csv file
pop_regions_df.to_csv('data/processed/population/population-region.csv', sep=';', index=False, header=True)

#### Save population countries dataset

In [77]:
columns_country = ['country_code', 'country_name', 'year', 'population']

# select columns
pop_country_df = pop_country_region_df[columns_country]

#store in csv file
pop_country_df.to_csv('data/processed/population/population-country.csv', sep=';', index=False, header=True)

#### Compare calculated region population to gapminder dataset

In [78]:
regions_population_gapminder = pop_dict.get('data-for-regions-by-year')
regions_population_2021_gapminder = regions_population_gapminder[regions_population_gapminder['time'] == 2021]
total_population_gapminder = regions_population_2021_gapminder['Population'].sum()

regions_population_2021_calc = pop_regions_df[pop_regions_df['year'] == 2021]
total_population_calc = regions_population_2021_calc['population'].sum()

print('Total population from the year 2021')
print('Gapminder ' + str(total_population_gapminder))
print('Calculated ' + str(total_population_calc))

print('\n')
print('Gapminder region population')
print(regions_population_2021_gapminder[['name', 'Population']])

print('\n')
print('Calculated region population')
print(regions_population_2021_calc[['region_name', 'population']])

Total population from the year 2021
Gapminder 7898737625.0
Calculated 1057364421


Gapminder region population
          name    Population
221     africa  1.391823e+09
522       asia  4.634610e+09
823     europe  8.460503e+08
1124  americas  1.026254e+09


Calculated region population
    region_name  population
121      Africa  1391823318
243     Oceania    43602426
365    Americas  1026253579
487        Asia -2147483648
609      Europe   743168746


As we can see, we get the same results for the total populations in the year 2021.
If we take a look at the region population, we see that the calculated population of america and africa shows no difference to the gapminder-dataset population. For the regions asia and europe, we get different results. Furthermore, the calculated region oceania does not exist in the gapminder dataset. Since the documentation of the gapminder dataset does not include the country-to-region assignment, we cannot compare the differences between our calculation and gapminder. However, since the overall score is equal, we can argue that those differences occur tue to different country-to-region assignments.